In [7]:
kittens = open('./example.in')

In [8]:
lines = kittens.readlines()

In [9]:
lines = [l.strip() for l in lines]
current_line = 0

In [10]:
videos_count, endpoints_count, requests_descriptions_count, caches_count, cache_size = [int(l) for l in lines[current_line].split(' ')]
current_line += 1

In [11]:
video_sizes = [int(l) for l in lines[current_line].split()]
current_line += 1

In [12]:
videos_map = dict()
for i in range(videos_count):
    videos_map[i] = video_sizes[i]

latencies = dict()

In [13]:
for ec in range(endpoints_count):
    endpoint_dc_latency, caches_amount = [int(l) for l in lines[current_line].split()]
    current_line += 1
    latencies[(ec, -1)] = endpoint_dc_latency

    for ca in range(caches_amount):
        cache_index, latency = [int(l) for l in lines[current_line].split()]
        current_line += 1

        latencies[(ec, cache_index)] = latency


In [14]:
requests_list = []
for i in range(requests_descriptions_count):
    video_id, endpoint_id, requests_count = [int(l) for l in lines[current_line].split()]
    current_line += 1

    requests_list.append((video_id, endpoint_id, requests_count))

In [15]:
requests_list

[(3, 0, 1500), (0, 1, 1000), (4, 0, 500), (1, 0, 1000)]

In [16]:
endpoints_to_sources = dict()

for (endpoint, source_id),v in latencies.items():
    if endpoint in endpoints_to_sources:
        endpoints_to_sources[endpoint].add(source_id)
    else:
        endpoints_to_sources[endpoint] = set([source_id])

In [17]:
source2videos = {0: set([2]), 1: set([3, 1]), 2: set([0, 1])}

In [22]:
def score(videos_endpoints_requests):
    score = 0
    for video, endpoint, request in videos_endpoints_requests:
        sources = endpoints_to_sources[endpoint]
        lats = []
        for source in sources:
            if source != -1 and video in source2videos[source]:
                lats.append(latencies[(endpoint, source)])
            else: 
                lats.append(latencies[(endpoint, -1)])
        score += request * (latencies[(endpoint, -1)] - min(lats))
    return score

In [23]:
score(requests_list)

1850000